In [1]:
import json
import pandas as pd
import numpy as np
import json
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from collections import Counter
from bert_embedding import BertEmbedding
from operator import itemgetter
import tensorflow as tf
import tensorflow_hub as hub
import spacy
import gensim.downloader as api
from nltk.corpus import wordnet

In [2]:
def convert_lower_case(data):
    # Convert the input text into lowercase text
    return np.char.lower(str(data))

def remove_stop_words(data):
    # Tokenize the input text and remove stopwords from the corpus
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 3:
            new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text

def remove_punctuation(data):
    # Remove punctuations defined below from input text
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    # Remove apostrophe from the input text
    return np.char.replace(data, "'", "")

def convert_numbers(data):
    # Convert numbers to text form in input text
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def get_bigrams(text):
    
    """Input
    ----------
    text : str or list of strings
    n    : number of word in each combination string ie if n = 2 the tokenization will happen in two word pairs
    
    Output
    -------
    tokens : The output would be a list of lists and each element list of the list will contain
             unigram and n_gram tokens. This functions can be modified for a range of grams but right now
             it will be best to use it with n = 2.
    """
    bi_grams = ngrams(word_tokenize(text), 2)
    unigrams = word_tokenize(text)
    bigrams = [' '.join(grams) for grams in bi_grams]
    tokens = unigrams + bigrams
    return tokens

def preprocess(data):
    # Preprocess the input text
    data = convert_lower_case(data)
    #data = remove_punctuation(data) #remove comma seperately
    #data = remove_apostrophe(data)
    data = remove_stop_words(data)
    #data = convert_numbers(data)
    #data = remove_punctuation(data)
    #data = convert_numbers(data)
    #data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    #data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [3]:
def get_tokens(dataframe, column):
    tokens = []
    for i in dataframe[column]:
        tokens.append(get_bigrams(preprocess(str(i))))
    return tokens

In [4]:
model = api.load('word2vec-google-news-300') # loading the Word2Vec pretrained embeddings (size = 1.6 GB)

KeyboardInterrupt: 

In [4]:
def get_synonyms(word):
    # get a list of all the synonyms of the input word
    word = wordnet.synsets(word)
    syns = [w.lemma_names() for w in word]
    synonyms = []
    for i in syns:
        for j in i:
            synonyms.append(j)
    synonyms = unique(synonyms)
    return synonyms

In [5]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

In [6]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [30]:
Title = []
Abstract = []
Url = []

path = 'C://Users//HP//Desktop//Articles//'
file_name = '_Article.json'

for i in range(1, 11):
    with open(path+str(i)+file_name, encoding="utf8") as json_file:
        temp_data = json.load(json_file)
        for j in temp_data:
            temp_title = j['Title']
            temp_abstract = j['Abstract']
            temp_url = j['Url']
            Title.append(temp_title)
            Abstract.append(temp_abstract)
            Url.append(temp_url)
            
df = pd.DataFrame()
df['Title'] = Title
df['Abstract'] = Abstract
df['Url'] = Url
df

,Title,Abstract,Url
0,Structural performance of non-plastered modula...,Straw bale construction has a proven track rec...,https://www.sciencedirect.com/science/article/...
1,Effects of roll and flail conditioning systems...,Miscanthus and switchgrass have been identifie...,https://www.sciencedirect.com/science/article/...
2,Effectiveness of straw bale check dams at redu...,Post-fire flooding and elevated sediment loads...,https://www.sciencedirect.com/science/article/...
3,Construction and monitoring of experimental st...,Straw bale buildings have the potential reduce...,https://www.sciencedirect.com/science/article/...
4,A review of densified solid biomass for energy...,Growing concerns over the environmental impact...,https://www.sciencedirect.com/science/article/...
...,...,...,...
37927,Glossary,,https://www.sciencedirect.com/science/article/...
37928,Glossary,,https://www.sciencedirect.com/science/article/...
37929,e-Glossary,,https://www.sciencedirect.com/science/article/...
37930,Glossary,,https://www.sciencedirect.com/science/article/...


In [9]:
def tfidf_scores(abstract, title):
    
    """given body and titles of the articles this funtion calculates the tfidf scores of the words in the text corpus.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       title        : Titles of the articles
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary.
       """
   
    
    N = len(abstract)
    DF = {}

    for i in range(N):
        tokens = abstract[i]
        for w in tokens:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]

    doc = 0

    tf_idf = {}

    for i in range(N):
    
        tokens = abstract[i]
    
        counter = Counter(tokens + title[i])
        words_count = len(tokens + title[i])
    
        for token in np.unique(tokens):
        
            tf = counter[token]/words_count
            df = doc_freq(token)
            idf = np.log((N+1)/(df+1))
        
            tf_idf[doc, token] = tf*idf

        doc += 1
    return tf_idf



def tfidf_abstract_scores(abstract):
    
    """given body articles this funtion calculates the tfidf scores of the words in the text corpus.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary.
       """
    
    
    N = len(abstract)
    DF = {}

    for i in range(N):
        tokens = abstract[i]
        for w in tokens:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]

    doc = 0

    tf_idf = {}

    for i in range(N):
    
        tokens = abstract[i]
    
        counter = Counter(tokens)
        words_count = len(tokens)
    
        for token in np.unique(tokens):
        
            tf = counter[token]/words_count
            df = doc_freq(token)
            idf = np.log((N+1)/(df+1))
        
            tf_idf[doc, token] = tf*idf

        doc += 1
    return tf_idf


def tfidf_title_scores(title):
    
    """given body articles this funtion calculates the tfidf scores of the words in the text corpus.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary.
       """
    N = len(title)
    DF = {}

    for i in range(N):
        tokens = title[i]
        for w in tokens:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]

    doc = 0

    tf_idf = {}

    for i in range(N):
    
        tokens = title[i]
    
        counter = Counter(tokens)
        words_count = len(tokens)
    
        for token in np.unique(tokens):
        
            tf = counter[token]/words_count
            df = doc_freq(token)
            idf = np.log((N+1)/(df+1))
        
            tf_idf[doc, token] = tf*idf

        doc += 1
    return tf_idf

In [10]:
def tfidf_title_filter(query, title, k):
    """given the query, body of articles, titles of articles, k (output articles threshold) and tfifd_method
      this funtion extract k number of articles that are most relevent to the query keywords.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       title        : Titles of the articles
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary."""
    tokens = query
    

    tf_idf = tfidf_title_scores(title)
    
    relevent_indices = []
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)    
    
    for i in query_weights[:k]:
        relevent_indices.append(i[0])
        
    return relevent_indices

def tfidf_abstract_filter(query, abstract, k):
    """given the query, body of articles, titles of articles, k (output articles threshold) and tfifd_method
      this funtion extract k number of articles that are most relevent to the query keywords.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       title        : Titles of the articles
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary."""
    tokens = query
    
    tf_idf = tfidf_abstract_scores(abstract)
    
    relevent_indices = []
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)    
    
    for i in query_weights[:k]:
        relevent_indices.append(i[0])
        
    return relevent_indices


def tfidf_abstract_title_filter(query, abstract, title,  k):
    """given the query, body of articles, titles of articles, k (output articles threshold) and tfifd_method
      this funtion extract k number of articles that are most relevent to the query keywords.
       
       Input
       -------
       body         : Body or abstracts of the articles.
       title        : Titles of the articles
       
       Output
       -------
       tf_idf       : A dictionary of tf_idf scorese of the vocabulary."""
    tokens = query
    
    tf_idf = tfidf_scores(abstract, title)
    
    relevent_indices = []
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)    
    
    for i in query_weights[:k]:
        relevent_indices.append(i[0])
        
    return relevent_indices

In [11]:
def Word_Mover_Distance(query, docs, k):
    
    """given a query, corpus of text, a list of indices, and k, this functions outputs a list of indices
       of the documents that are most similar to the query based on the context captured by context captured by
       Word2vec pretrained embeddings "Google-news-300" It’s 1.5GB! It includes word vectors for a vocabulary
       of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset.
       The vector length is 300 features. This function use Word Mover Distance to calculate the similarity
       between documents and given query.
       
       Input
       -------
       query          : A string of words.
       docs           : Text corpus, a list of lists with tokenized sentences.
       indices        : A list of indices.
       k              : Number of outputs.
       
       Output
       -------
       sorted_indices : A list of k number of indices belong the documents that are most relevent to query.
       """
    
    relevent_docs = []
    for i in range(len(docs)):
        relevent_docs.append(docs[i])
        
    distance = []
    for doc in relevent_docs:
        distance.append(model.wmdistance(doc, query))
    
    docs_distance = []
    for i in range(len(docs)):
        docs_distance.append([distance[i], i])
    
    sorted_docs_distance = sorted(docs_distance, key=itemgetter(0), reverse = False)
    sorted_indices = []
    for i in sorted_docs_distance[:k]:
        sorted_indices.append(i[1])
    return sorted_indices

In [20]:
def nlp_filter_regional(dataframe, column):
    '''Given data frame and column of the data frame this function outputs the indices of articles that do not have
        the name of any goepolical entity in it.
       
       Input
       -------
       dataframe    : A pandas data frame.
       column       : Specific column of the data frame.
       
       Output
       -------
       indices      : A list of indices of articles that does not have have any geoplolictacl entity in them.'''
    
    nlp = spacy.load('en_core_web_sm')
    indices = []
    doc_ents = []
    for i in range(len(dataframe[column])):
        document = dataframe.loc[i, column]
        document = remove_stop_words(document)
        document = word_tokenize(document)
        _doc_ents = []
        for word in document:
            word = nlp(word)
            for entity in word.ents:
                _doc_ents.append(entity.label_)
        doc_ents.append(_doc_ents)

    for i in range(len(dataframe[column])):
        doc = doc_ents[i]
        if 'GPE' not in doc:
            indices.append(i)
            
        else:
            print('Aritcle', i, 'has a Geopolitical Entity in it')
    return indices

In [21]:
def nlp_filter_historical(dataframe, column):
    
    """Given data frame and column of the data frame this function outputs the indices of articles that do not have
        the both the date and comparision words in it.
       
       Input
       -------
       dataframe    : A pandas data frame.
       column       : Specific column of the data frame.
       
       Output
       -------
       indices      : A list of indices of articles that does not have both the date and comparision word in them.
       """
    nlp = spacy.load('en_core_web_sm')
    indices = []
    doc_ents = []
    for i in range(len(dataframe[column])):
        document = dataframe.loc[i, column]
        document = remove_stop_words(document)
        document = word_tokenize(document)
        _doc_ents = []
        for word in document:
            word = nlp(word)
            for entity in word.ents:
                _doc_ents.append(entity.label_)
        doc_ents.append(_doc_ents)

    for i in range(len(dataframe[column])):
        doc = doc_ents[i]
        comperision = ['from', 'since', 'versus', 'vs', 'between']
        if 'DATE' in doc:
            document = dataframe.loc[i, column]
            document = word_tokenize(document)
            for token in document:
                if token in comperision:
                    print('Aritcle', i, 'has a date and a comperision word')
        else:
            indices.append(i)
    return indices

In [14]:
def keywords_filter(query, dataframe, col_name, top_k,similarity_method):
    
    """given query, dataframe, column name or top k numbers of output and similarity method as input this 
       function extract articles that are most relevent to the query keywords by using tfidf model and then
       sort these relevent articles based on contextual similarity by any of 4 state of the art methods for
       semantic similarity of text. Finaly it outputs a data frame that contains the title, abstract and url
       of the relevent article.
       
       Input
       -------
       query          : A string of words(keywords to match the articles).
       dataframe      : A pandas data frame.
       col_name       : Input either 'Abstract' or 'Title' 'Abstract+Title' for ranking.
       top_k          : Number of filtered articles.
       method         : One of four otiopns for semantic similarity 'cosine_similarity', 'universal_encoder_inner',
                        'universal_encoder_cosine' and 'word_mover_distance'.
       
       Output
       -------
       indices        : A list of indices that belong to k number of most relavent documents.
       """
    
    if col_name == 'Abstract':
        docs = get_tokens(dataframe, col_name)
        if similarity_method == 'tfidf':
            final_indices = tfidf_abstract_filter(query, docs,  top_k)
        elif similarity_method == 'word_mover_distance':
            final_indices = Word_Mover_Distance(query, docs, top_k)
        else:
            pass
        
    elif col_name == 'Title':
        docs = get_tokens(dataframe, col_name)
        if similarity_method == 'tfidf':
            final_indices = tfidf_title_filter(query, docs, top_k)
        elif similarity_method == 'word_mover_distance':
            final_indices = Word_Mover_Distance(query, docs, top_k)
        else:
            pass
        
    elif col_name == 'Abstract+Title':
        abstract = get_tokens(dataframe, 'Abstract')
        title = get_tokens(dataframe, 'Title')
        if similarity_method == 'tfidf':
            final_indices = tfidf_abstract_title_filter(query, abstract, title, top_k)
        elif similarity_method == 'word_mover_distance':
            final_indices = Word_Mover_Distance(query, abstract, top_k)
        else:
            pass
    
    return final_indices

In [15]:
def nlp_filter(query, query_method, dataframe, filter_type, col_name, top_k, similarity_method):
    
    
    """given query, query method, dataframe, filter_type, column name or top k numbers of outputs and similarity
       method as input this function extract articles that are most relevent to the query keywords by using tfidf
       model and then sort these relevent articles based on contextual similarity by any of 4 state of the art methods for
       semantic similarity of text. Finaly it outputs a data frame that contains the title, abstract and url
       of the relevent article.
       
       Input
       -------
       query          : A string of words(keywords to match the articles).
       query_method   : Method of query either normal 'query' or 'scope'
       dataframe      : A pandas data frame.
       filter_type    : Chose from options "Keywords", "Historical", "Regional"
       col_name       : Input either 'Abstract' or 'Title' 'Abstract+Title' for ranking.
       top_k          : Number of filtered articles.
       method         : One of four otiopns for semantic similarity 'cosine_similarity', 'universal_encoder_inner',
                        'universal_encoder_cosine' and 'word_mover_distance'.
       
       Output
       -------
       indices        : A list of indices that belong to k number of most relavent documents.
       """

    
    if query_method == 'query':
        query = get_bigrams(preprocess(query))
    elif query_method == 'scope':
        query = get_synonyms(query)
    else:
        pass
    
    if filter_type == 'Keywords':
        final_indices = keywords_filter(query, dataframe, col_name, top_k, similarity_method)
        
    elif filter_type == 'Regional':
        if col_name == 'Title':
            col_name = col_name
        elif col_name == 'Abstract':
            col_name = col_name
        elif col_name == 'Abstract+Title':
            col_name = 'Abstract'
        else:
            pass
        final_indices = nlp_filter_regional(dataframe, col_name)
        
    elif filter_type == 'Historical':
        if col_name == 'Title':
            col_name = col_name
        elif col_name == 'Abstract':
            col_name = col_name
        elif col_name == 'Abstract+Title':
            col_name = 'Abstract'
        else:
            pass
        final_indices = nlp_filter_historical(dataframe, col_name)
    else:
        pass
    return final_indices

## Example - query_method  'scope':

If the user select the query_method 'scope' the user can input a single word as query, the query would then be converted into a list of synonyms and used as query in the algorithm.

In [25]:
indices = nlp_filter('aqua', 'scope', df, 'Keywords', 'Abstract', 10, 'word_mover_distance')

In [26]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Before the beginning,The very first living thing is still alive ins...
1,Traitement conventionnel du myélome multiple,Environ 50% des patients atteints de myélome m...
2,Dissolution behavior of silk fibroin in a low ...,Regenerated Silk biomaterials are usually pre-...
3,Chapter 12: Olea europaea as Potential Source ...,The olive tree (Olea europaea L.) belongs to t...
4,Chapter 12: Olea europaea as Potential Source ...,The olive tree (Olea europaea L.) belongs to t...
5,Water hyacinths as a resource in agriculture a...,Water hyacinths are becoming a problem in lake...
6,"Chapter 32: City of San Francisco, California,...",Located on a peninsula in northern California ...
7,Designing for Workspace Safety,Abstract
8,Chapter 3: Wastewater remediation via combo-te...,Abstract
9,Exploiting microbubble-microbe synergy for bio...,Abstract


## Example - query_method  'query':

If the user select query_method as 'query' the user can input multiple words as query, these words would then be converted into a list of unigram and bigram tokens and used in the algorithm to extract relevent articles.

In [28]:
indices = nlp_filter('climate change', 'query', df, 'Keywords', 'Abstract', 10, 'word_mover_distance')

In [29]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Chapter 13: Climate Change and Agricultural Sy...,In this chapter we outline the two-way links b...
1,Chapter 21: Cyanobacterial Bioenergy and Biofu...,"In recent decades, there have been significant..."
2,Variation in annual carbon fluxes affecting th...,Estimation of soil-related carbon (C) fluxes i...
3,Chemicals From Renewable Sources,Increasing concerns on climate change and ener...
4,Plants in a warmer world,Climate is a major determinant for the phenolo...
5,"A review of climate change, mitigation and ada...",Global climate change is a change in the long-...
6,Chapter Two: Global Warming and Its Possible I...,Progress has been significant in climate scien...
7,Chapter Two: Global Warming and Its Possible I...,Progress has been significant in climate scien...
8,Modeling of Thermochemical Conversion of Bioma...,"The growth in the world energy demand, as well..."
9,Decadal analysis of impact of future climate o...,"Different aspects of climate change, such as i..."


## Example - Filter Type 'Historical':

If user want to filter out all the articles that have historical comperision in them he/she can type 'Historical' in filter_type.

In [24]:
indices = nlp_filter('biomass', 'query', df, 'Historical', 'Abstract', 10, 'word_mover_distance')

Aritcle 2 has a date and a comperision word
Aritcle 2 has a date and a comperision word
Aritcle 4 has a date and a comperision word
Aritcle 4 has a date and a comperision word
Aritcle 6 has a date and a comperision word
Aritcle 7 has a date and a comperision word
Aritcle 13 has a date and a comperision word
Aritcle 19 has a date and a comperision word
Aritcle 19 has a date and a comperision word
Aritcle 19 has a date and a comperision word
Aritcle 28 has a date and a comperision word
Aritcle 33 has a date and a comperision word
Aritcle 33 has a date and a comperision word
Aritcle 33 has a date and a comperision word
Aritcle 33 has a date and a comperision word
Aritcle 36 has a date and a comperision word
Aritcle 36 has a date and a comperision word
Aritcle 36 has a date and a comperision word
Aritcle 37 has a date and a comperision word
Aritcle 37 has a date and a comperision word
Aritcle 37 has a date and a comperision word
Aritcle 64 has a date and a comperision word
Aritcle 67 has a

In [25]:
indices

[0,
 1,
 3,
 5,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 29,
 30,
 31,
 32,
 34,
 35,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 80,
 81,
 84,
 86,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 106,
 107,
 108,
 109,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 127,
 128,
 129,
 130,
 133,
 134,
 135,
 138,
 139,
 140,
 141,
 142,
 146,
 148,
 149,
 153,
 154,
 156,
 158,
 159,
 160,
 161,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 185,
 186,
 187,
 189,
 190,
 191,
 192,
 193,
 195,
 196,
 197,
 198,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 216,
 217,
 219,
 220,
 221,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 233,
 238,
 239,
 241,
 242,
 243,
 2

## Example - Filter Type 'Regional':

If user want to filter out all the articles that have name of a Geo Political Entity in them he/she can type 'Regional' in filter_type.

In [18]:
indices = nlp_filter('biomass', 'query', df, 'Regional', 'Abstract', 10, 'word_mover_distance')

Aritcle 1 has a Geopolitical Entity in it
Aritcle 2 has a Geopolitical Entity in it
Aritcle 3 has a Geopolitical Entity in it
Aritcle 6 has a Geopolitical Entity in it
Aritcle 9 has a Geopolitical Entity in it
Aritcle 13 has a Geopolitical Entity in it
Aritcle 14 has a Geopolitical Entity in it
Aritcle 16 has a Geopolitical Entity in it
Aritcle 19 has a Geopolitical Entity in it
Aritcle 20 has a Geopolitical Entity in it
Aritcle 33 has a Geopolitical Entity in it
Aritcle 35 has a Geopolitical Entity in it
Aritcle 37 has a Geopolitical Entity in it
Aritcle 42 has a Geopolitical Entity in it
Aritcle 46 has a Geopolitical Entity in it
Aritcle 50 has a Geopolitical Entity in it
Aritcle 53 has a Geopolitical Entity in it
Aritcle 60 has a Geopolitical Entity in it
Aritcle 64 has a Geopolitical Entity in it
Aritcle 74 has a Geopolitical Entity in it
Aritcle 76 has a Geopolitical Entity in it
Aritcle 79 has a Geopolitical Entity in it
Aritcle 82 has a Geopolitical Entity in it
Aritcle 84 has a

In [22]:
indices

[0,
 4,
 5,
 7,
 8,
 10,
 11,
 12,
 15,
 17,
 18,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 36,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 47,
 48,
 49,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 61,
 62,
 63,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 75,
 77,
 78,
 80,
 81,
 83,
 85,
 87,
 90,
 91,
 92,
 95,
 96,
 97,
 98,
 99,
 102,
 103,
 104,
 106,
 108,
 111,
 113,
 114,
 117,
 120,
 121,
 123,
 126,
 128,
 130,
 132,
 133,
 141,
 147,
 150,
 153,
 159,
 160,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 186,
 187,
 190,
 191,
 192,
 195,
 196,
 197,
 198,
 199,
 200,
 202,
 203,
 204,
 205,
 206,
 207,
 209,
 210,
 212,
 213,
 214,
 215,
 216,
 217,
 219,
 220,
 221,
 222,
 223,
 224,
 226,
 228,
 230,
 231,
 233,
 234,
 235,
 238,
 241,
 242,
 243,
 245,
 246,
 248,
 249,
 250,
 253,
 255,
 256,
 259,
 261,
 262,
 264,
 265,
 266,
 268,
 269,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 2

## Example - col_name 'Abstract':

If user select 'Abstract' in col_name then both tfidf and word_mover_distance methods will perform their calculations on Abstract.

In [37]:
indices = nlp_filter('genetic engineering', 'query', df, 'Keywords', 'Abstract', 10, 'tfidf')

In [38]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Chapter 11: Fundamentals of Biochemical Reacti...,Biochemical reaction engineering is a branch o...
1,Genetic engineering of wood,The technology of genetic engineering provides...
2,Biodiesel production from genetically engineer...,Current biomass sources for energy production ...
3,Chapter 1: Genetic Engineered Organisms (Plant...,Genetically modified organism (GMO) is one who...
4,Genetic Manipulation of Non-Classic Oilseed Pl...,Global demand for vegetable oil is anticipated...
5,Chapter 16: Engineering Lessons—Using Engineer...,Engineering plays a critical role in most aspe...
6,Cellulolytic and xylanolytic enzymes of thermo...,Thermophilic microorganisms are ubiquitous in ...
7,Cellulolytic and xylanolytic enzymes of thermo...,Thermophilic microorganisms are ubiquitous in ...
8,Programming Cell-Cell Interactions through Non...,The ability to direct targeted intercellular i...
9,Chapter 2: Metabolic Engineering and Genetic M...,Agricultural biomass offers one of the best pl...


## Example - col_name 'Title':

If user select 'Title' in col_name then both tfidf and word_mover_distance methods will perform their calculations on Title.

In [39]:
indices = nlp_filter('genetic engineering', 'query', df, 'Keywords', 'Title', 10, 'tfidf')

In [40]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Genetic engineering of wood,The technology of genetic engineering provides...
1,Genetic engineering in biomimetic composites,Composites represent a class of materials with...
2,Biochemical engineering,
3,Biochemical engineering,
4,Biochemical engineering,
5,Advances in genetic engineering of marine algae,Algae are a component of bait sources for anim...
6,Biochemical engineering,
7,Genetic engineering of E. coli SE5000 and its ...,A genetically engineered Escherichia coli SE50...
8,2.58: Biorefinery Engineering,Biorefinery that utilizes renewables for produ...
9,Engineering aspects of ensiling,Ensiling is a preservation method for moist fo...


## Example - col_name 'Title':

If user select 'Abstract+Title' in col_name then tfidf will calculate relevence on both Abstracts and Titles while word_mover_distance will only calculate the relevence on Abstracts

In [41]:
indices = nlp_filter('genetic engineering', 'query', df, 'Keywords', 'Abstract+Title', 10, 'tfidf')

In [42]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Chapter 11: Fundamentals of Biochemical Reacti...,Biochemical reaction engineering is a branch o...
1,Genetic engineering of wood,The technology of genetic engineering provides...
2,Chapter 16: Engineering Lessons—Using Engineer...,Engineering plays a critical role in most aspe...
3,Genetic engineering in biomimetic composites,Composites represent a class of materials with...
4,Chapter 1: Genetic Engineered Organisms (Plant...,Genetically modified organism (GMO) is one who...
5,Chemistry and engineering of the production pr...,The present paper discusses the evolution of t...
6,Chapter 2: Metabolic Engineering and Genetic M...,Agricultural biomass offers one of the best pl...
7,Lignin genetic engineering for improvement of ...,"Lignin, a complex racemic phenolic heteropolym..."
8,Lignin genetic engineering for improvement of ...,"Lignin, a complex racemic phenolic heteropolym..."
9,Chapter 14: Genetic Engineering Applications t...,Cellulase production with superior expertise i...


## Example - similarity_method 'tfidf':

If the user select similarity_method 'tfidf' the function will return the list of indices of the k most relevent articles to the query based on word frequencies. 

In [32]:
indices = nlp_filter('biomass', 'query', df, 'Keywords', 'Abstract', 10, 'tfidf')

In [33]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Chapter 3: Biomass Torrefaction Process,This chapter presents an overview of the torre...
1,Phosphorus recovery from the biomass ash: A re...,"Biomass ash, generated during the thermal chem..."
2,Phosphorus recovery from the biomass ash: A re...,"Biomass ash, generated during the thermal chem..."
3,Chapter 3: Biomass Characteristics,The characteristics of biomass greatly influen...
4,Hydrothermal conversion of biomass to fuels an...,"Available biomass, preferentially residues, ca..."
5,A review on exergy analysis of biomass based f...,Renewable energy sources can be a good substit...
6,Biomass supply chain optimisation via novel Bi...,Negative environment impacts and security issu...
7,The co-combustion of hard coal with raw and to...,The co-combustion of raw and torrefied biomass...
8,Potential of cofiring with biomass in Italy,Biomass is considered a potential fuel and a r...
9,4: Biomass feedstock for IGCC systems,Biomass gasification integrated with a gas tur...


## Example - similarity_method 'word_mover_distance':
If the user select similarity_method 'word_mover_distance' the function will return the list of indices of the k most relevent articles to the query based on word frequencies

In [34]:
indices = nlp_filter('biomass', 'query', df, 'Keywords', 'Abstract', 10, 'word_mover_distance')

In [35]:
title = []
abstract = []
for i in indices:
    title.append(df.loc[i, 'Title'])
    abstract.append(df.loc[i, 'Abstract'])
data = pd.DataFrame()
data['Title'] = title
data['Abstract'] = abstract
data

,Title,Abstract
0,Chapter 3: Biomass Torrefaction Process,This chapter presents an overview of the torre...
1,Phosphorus recovery from the biomass ash: A re...,"Biomass ash, generated during the thermal chem..."
2,Phosphorus recovery from the biomass ash: A re...,"Biomass ash, generated during the thermal chem..."
3,A review on exergy analysis of biomass based f...,Renewable energy sources can be a good substit...
4,4: Biomass feedstock for IGCC systems,Biomass gasification integrated with a gas tur...
5,The co-combustion of hard coal with raw and to...,The co-combustion of raw and torrefied biomass...
6,Hydrothermal conversion of biomass to fuels an...,"Available biomass, preferentially residues, ca..."
7,Chapter 3: Biomass Characteristics,The characteristics of biomass greatly influen...
8,An Assessment of the Effects of Biomass Proces...,Biomass can be cofired with coal in different ...
9,Potential of cofiring with biomass in Italy,Biomass is considered a potential fuel and a r...
